#Run custom functions in LangChain
Made by: Wilfredo Aaron Sosa Ramos (AI Lab Manager at RealityAI Labs)

##1. Install dependencies

In [1]:
!pip install -q langchain langchain_core langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00


##2. Set env. variables

In [2]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

##3. With the constructor

In [7]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

# Accountability-specific function
def accountability_score(task_completions, deadlines_met):
    """Calculate accountability score based on tasks completed and deadlines met."""
    if deadlines_met == 0:  # Prevent division by zero
        return 0
    return task_completions / deadlines_met

# Economics Engineering-specific function
def economic_efficiency(revenue, costs):
    """Calculate economic efficiency as the ratio of revenue to costs."""
    if costs == 0:  # Prevent division by zero
        return float('inf')  # Infinite efficiency if costs are zero
    return revenue / costs

# Nested function for combining both fields
def combined_analysis(data):
    """Combine accountability and economic efficiency into a single score."""
    accountability = accountability_score(
        data["task_completions"], data["deadlines_met"]
    )
    efficiency = economic_efficiency(data["revenue"], data["costs"])
    return {"combined_score": accountability * efficiency}

# Instantiate the model
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp')

# Create a prompt template
prompt = ChatPromptTemplate.from_template("Performance Analysis:\nAccountability Score: {a}\nEconomic Efficiency: {b}")

# Define the chains
chain = (
    {
        "a": {
            "task_completions": itemgetter("tasks_done"),
            "deadlines_met": itemgetter("deadlines"),
        } | RunnableLambda(lambda x: {"accountability_score": accountability_score(x["task_completions"], x["deadlines_met"])}),
        "b": {
            "revenue": itemgetter("revenue"),
            "costs": itemgetter("costs"),
        } | RunnableLambda(lambda x: {"economic_efficiency": economic_efficiency(x["revenue"], x["costs"])}),
    }
    | prompt
    | model
)

# Combined chain for advanced analysis
advanced_chain = (
    {
        "task_completions": itemgetter("tasks_done"),
        "deadlines_met": itemgetter("deadlines"),
        "revenue": itemgetter("revenue"),
        "costs": itemgetter("costs"),
    }
    | RunnableLambda(combined_analysis)
    | ChatPromptTemplate.from_template("Advanced Performance Analysis:\nCombined Score: {combined_score}")
    | model
)

In [8]:
result = chain.invoke({
    "tasks_done": 50,
    "deadlines": 10,
    "revenue": 2000,
    "costs": 500
})

advanced_result = advanced_chain.invoke({
    "tasks_done": 50,
    "deadlines": 10,
    "revenue": 2000,
    "costs": 500
})

print("Result:", result)
print("Advanced Analysis Result:", advanced_result)

Result: content='Okay, let\'s break down this performance analysis:\n\n**Accountability Score: {\'accountability_score\': 5.0}**\n\n* **Interpretation:** This indicates a very high level of accountability. A score of 5.0 (assuming a scale where 5 is the highest) suggests that the entity or individual being assessed is consistently and thoroughly taking responsibility for their actions, decisions, and outcomes.\n* **Positive Implications:**\n    * **Strong Ownership:** There\'s a clear sense of ownership over tasks and responsibilities.\n    * **Reliability:** The entity is likely reliable and dependable in fulfilling commitments.\n    * **Transparency:** High accountability often implies transparency in processes and reporting.\n    * **Potential for Improvement:**  Accountability provides a solid foundation for identifying and addressing areas for improvement.\n\n**Economic Efficiency: {\'economic_efficiency\': 4.0}**\n\n* **Interpretation:** This signifies a good level of economic ef

In [9]:
!pip -q install -q rich

In [10]:
from rich.console import Console
from rich.markdown import Markdown

def print_md(md_content):
    """Renders and prints Markdown content using the `rich` library."""
    console = Console()
    md = Markdown(md_content)
    console.print(md)

In [12]:
print("=====RESULT=====")
print_md(result.content)

=====RESULT=====


Okay, let's break down this performance analysis:                                                                  

Accountability Score: {'accountability_score': 5.0}                                                                

 • Interpretation: This indicates a very high level of accountability. A score of 5.0 (assuming a scale where 5 is 
   the highest) suggests that the entity or individual being assessed is consistently and thoroughly taking        
   responsibility for their actions, decisions, and outcomes.                                                      
 • Positive Implications:                                                                                          
    • Strong Ownership: There's a clear sense of ownership over tasks and responsibilities.                        
    • Reliability: The entity is likely reliable and dependable in fulfilling commitments.                         
    • Transparency: High accountability often implies transparency in processes and reporting.                     
    • Potential for Improvement:  Accountability provides a solid foundation for identifying and addressing areas  
      for improvement.                                                                                             

Economic Efficiency: {'economic_efficiency': 4.0}                                                                  

 • Interpretation: This signifies a good level of economic efficiency. A score of 4.0 (again, assuming a scale     
   where 5 is the highest) suggests that resources are being used effectively and with a strong return on          
   investment.                                                                                                     
 • Positive Implications:                                                                                          
    • Resource Optimization:  The entity is generally good at minimizing waste and maximizing output with the      
      available resources (financial, human, materials, etc.).                                                     
    • Cost-Effectiveness: Operations are likely cost-effective and provide value for money.                        
    • Productivity:  There's a good level of productivity in terms of the ratio of inputs to outputs.              
 • Areas for Potential Improvement:                                                                                
    • Pushing for Optimization: While good, there's still room to strive for the highest level of efficiency (5.0).
      Could processes be streamlined further? Are there any untapped opportunities for cost reduction or resource  
      reallocation?                                                                                                
    • Continuous Monitoring: Regular monitoring of efficiency metrics is crucial to maintain and improve the       
      current level.                                                                                               

Overall Analysis and Potential Insights:                                                                           

 • Strengths: The analysis highlights strong accountability and a solid level of economic efficiency. These are    
   crucial for success in most contexts.                                                                           
 • Balanced Performance: The entity or individual is performing well in key areas.                                 
 • Focus Areas: While both scores are good, the economic efficiency score indicates a slight opportunity for       
   further improvement. This could be a good area to focus on if the goal is to achieve peak performance.          
 • Need for Context:  It's important to remember that these scores are likely based on specific criteria and a     
   specific context. To get a full understanding, it would be useful to know:                                      
    • The specific definition of "accountability" 

In [13]:
print("=====ADVANCED ANALYSIS RESULT=====")
print_md(advanced_result.content)

=====ADVANCED ANALYSIS RESULT=====


Okay, a Combined Score of 20.0 suggests a very high level of performance. To provide a more detailed analysis, we  
need context. However, based on the score alone, here's what we can infer and some questions we should ask:        

Inferences Based on a Score of 20.0:                                                                               

 • Exceptional Performance: A score of 20.0 is likely at or near the top of the possible scoring range. This       
   indicates that the individual, team, system, or whatever is being evaluated is performing exceptionally well.   
 • Significant Strengths: This score suggests that whatever is being assessed possesses significant strengths and  
   likely excels in most areas.                                                                                    
 • Potential for Mastery: The performance level is indicative of a high degree of mastery and competence.          
 • Possible Benchmark: Such a high score could be used as a benchmark for others to strive towards.                

Questions to Ask for a Deeper Analysis:                                                                            

To truly understand the significance of this score, we need to know:                                               

 1 What is being measured?                                                                                         
    • Is it an individual's performance on a specific task?                                                        
    • Is it a team's output on a project?                                                                          
    • Is it a system's efficiency?                                                                                 
    • What is the specific domain or field this score applies to?                                                  
    • Is it a game, a business metric, a scientific experiment, or something else?                                 
 2 What is the scoring scale?                                                                                      
    • What is the minimum and maximum possible score?                                                              
    • Is the scale linear, logarithmic, or another type?                                                           
    • What does each point on the scale represent in terms of performance?                                         
    • Is this a normalized score, or is it based on raw data?                                                      
 3 What are the individual components of the "Combined Score"?                                                     
    • What specific metrics are being combined to arrive at the final score?                                       
    • How are these metrics weighted?                                                                              
    • What are the individual scores for each component? Understanding the breakdown will reveal strengths and     
      potential areas for marginal improvement.                                                                    
 4 How does this score compare to others?                                                                          
    • Is this a record-breaking score?                                                                             
    • How does it compare to average performance in this field?                                                    
    • How does it compare to past performances of the same entity?                                                 
    • Is this score consistent, or is it an outlier?                                                               
 5 What are the context and circumstances?                                                                         
    • Were there any specific challenges or advantages present during the evaluation?                              
    • Were there any external factors that might ha

##4. Using the @chain decorator

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_google_genai import ChatGoogleGenerativeAI

# Define the prompts
prompt1 = ChatPromptTemplate.from_template(
    "Calculate the ROI (Return on Investment) given the following:\nRevenue: {revenue}\nCosts: {costs}\nProvide a detailed explanation."
)

prompt2 = ChatPromptTemplate.from_template(
    "Based on this financial summary, calculate the VAN (Net Present Value) and TIR (Internal Rate of Return):\nROI Explanation: {roi_explanation}\nAdditional details: {details}"
)

@chain
def roi_van_tir_chain(data):
    # Step 1: Calculate ROI
    prompt_val1 = prompt1.invoke({"revenue": data["revenue"], "costs": data["costs"]})
    output1 = ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp').invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)

    # Step 2: Calculate VAN and TIR
    chain2 = prompt2 | ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp') | StrOutputParser()
    return chain2.invoke({"roi_explanation": parsed_output1, "details": data["details"]})

In [15]:
result = roi_van_tir_chain.invoke({
    "revenue": 15000,
    "costs": 5000,
    "details": "The investment duration is 5 years with a 10% discount rate."
})

In [16]:
print_md(result)

Okay, let's calculate the Net Present Value (NPV) and Internal Rate of Return (IRR) based on the information       
provided.                                                                                                          

Understanding NPV and IRR                                                                                          

 • Net Present Value (NPV): NPV is the sum of the present values of all cash flows (both inflows and outflows)     
   associated with an investment. It tells you the net gain or loss in today's money terms from making the         
   investment. A positive NPV suggests a profitable investment, while a negative NPV suggests a loss.              
 • Internal Rate of Return (IRR): IRR is the discount rate at which the NPV of an investment equals zero. It       
   represents the expected rate of return on the investment. Generally, the higher the IRR, the more desirable the 
   investment.                                                                                                     

Given Information:                                                                                                 

 • Initial Investment (Cost): $5,000 (This is a cash outflow at time zero)                                         
 • Net Profit (Annual Cash Flow): $10,000 (This is assumed to be an annual inflow for 5 years)                     
 • Investment Duration: 5 years                                                                                    
 • Discount Rate: 10%                                                                                              

Calculations                                                                                                       

1. Net Present Value (NPV)                                                                                         

To calculate NPV, we need to discount each year's cash flow back to its present value and then sum them up,        
subtracting the initial investment.                                                                                

The formula for present value (PV) of a single cash flow is:                                                       

PV = CF / (1 + r)^n                                                                                                

Where:                                                                                                             

 • CF = Cash Flow                                                                                                  
 • r = Discount Rate                                                                                               
 • n = Year number                                                                                                 

Let's calculate the present value of each year's cash flow:                                                        

 • Year 1: $10,000 / (1 + 0.10)^1 = $9,090.91                                                                      
 • Year 2: $10,000 / (1 + 0.10)^2 = $8,264.46                                                                      
 • Year 3: $10,000 / (1 + 0.10)^3 = $7,513.15                                                                      
 • Year 4: $10,000 / (1 + 0.10)^4 = $6,830.13                                                                      
 • Year 5: $10,000 / (1 + 0.10)^5 = $6,209.21                                                                      

Now, let's calculate the NPV:                                                                                      

NPV = -Initial Investment + PV of Year 1 + PV of Year 2 + PV of Year 3 + PV of Year 4 + PV of Year 5               

NPV = -$5,000 + $9,090.91 + $8,264.46 + $7,513.15 + $6,830.13 + $6,209.21                                          

NPV = $32,907.86                                                                                                   

2. Internal Rate of Return (IRR)      

##5. Automatic coercion with chains

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a real case scenario about {economic_topic} in the context of Economics Engineering.")

# Initialize the model
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp')

# Chain with coerced function for Economics Engineering
chain_with_coerced_function = prompt | model | (lambda x: x.content.split('.')[0] + ".")

# Example invocation
result = chain_with_coerced_function.invoke({"economic_topic": "supply and demand dynamics"})

print("Result:", result)

Result: Okay, let's explore a real-world case scenario focusing on supply and demand dynamics in the context of Economics Engineering:

**Scenario: The Lithium-Ion Battery Boom and the Cobalt Shortage**

**Context:**

* **Product:** Lithium-ion batteries, crucial for electric vehicles (EVs), smartphones, laptops, and other portable electronic devices.


##6. Passing run metadata (Callbacks only for OpenAI)

In [18]:
!pip install -q langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.5 MB/s eta 0:00:00


In [19]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [20]:
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableConfig
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

# Initialize the model
model = ChatOpenAI(model='gpt-4o-mini')

# Function to parse or fix Economics Engineering-specific JSON data
def parse_or_fix_economics(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "The following text contains an Economics Engineering-related JSON structure "
            "with errors. Fix the data and ensure it adheres to the standards for "
            "Cost-Benefit Analysis (CBA) in public infrastructure projects:\n\n"
            "```text\n{input}\n```\nError: {error}\n"
            "Output the corrected JSON only, without explanations."
        )
        | model
        | StrOutputParser()
    )

    # Attempt to parse or fix the text up to 3 times
    for _ in range(3):
        try:
            return json.loads(text)  # Attempt to parse the JSON
        except Exception as e:
            # Invoke the fixing chain with the error details
            text = fixing_chain.invoke({"input": text, "error": str(e)}, config)
    return "Failed to parse after 3 attempts"

# Define a callback and invoke the parsing/fixing function
with get_openai_callback() as cb:
    faulty_data = '{"project_name": "Highway Expansion", "benefits": [500000, ,], "costs": [200000, "unknown"]}'

    output = RunnableLambda(parse_or_fix_economics).invoke(
        faulty_data,
        {"tags": ["EconomicsEngineering", "CBA"], "callbacks": [cb]},
    )

    print("Corrected Output:", output)
    print("Callback Metrics:", cb)

Corrected Output: Failed to parse after 3 attempts
Callback Metrics: Tokens Used: 440
	Prompt Tokens: 331
		Prompt Tokens Cached: 0
	Completion Tokens: 109
		Reasoning Tokens: 0
Successful Requests: 3
Total Cost (USD): $0.00011504999999999999


In [22]:
from pydantic import BaseModel, Field
from typing import List

class CostBenefitAnalysis(BaseModel):
    project_name: str = Field(..., title="Project Name", description="Name of the infrastructure project")
    benefits: List[float] = Field(
        ...,
        title="Benefits",
        description="List of monetary benefits (in USD) associated with the project",
        example=[500000, 200000],
    )
    costs: List[float] = Field(
        ...,
        title="Costs",
        description="List of monetary costs (in USD) associated with the project",
        example=[300000, 100000],
    )
    discount_rate: float = Field(
        ...,
        gt=0.0,
        le=1.0,
        title="Discount Rate",
        description="Discount rate (as a fraction, e.g., 0.1 for 10%) used for present value calculations",
        example=0.1,
    )
    project_duration: int = Field(
        ...,
        gt=0,
        title="Project Duration",
        description="Duration of the project in years",
        example=5,
    )
    net_present_value: float = Field(
        ...,
        title="Net Present Value",
        description="Net present value (NPV) of the project based on benefits and costs",
        example=500000.0,
    )
    internal_rate_of_return: float = Field(
        ...,
        gt=0.0,
        title="Internal Rate of Return",
        description="Internal rate of return (IRR) as a fraction (e.g., 0.15 for 15%)",
        example=0.15,
    )

In [24]:
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableConfig
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback
from langchain_core.output_parsers import JsonOutputParser

# Initialize the model
model = ChatOpenAI(model='gpt-4o-mini')

# Function to parse or fix Economics Engineering-specific JSON data
def parse_or_fix_economics(text: str, config: RunnableConfig):
    parser = JsonOutputParser(pydantic_object=CostBenefitAnalysis)
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "The following text contains an Economics Engineering-related JSON structure "
            "with errors. Fix the data and ensure it adheres to the standards for "
            "Cost-Benefit Analysis (CBA) in public infrastructure projects:\n\n"
            "```text\n{input}\n```\nError: {error}\n"
            "Output the corrected JSON only, without explanations."
            "This is the expected format: {format_instructions}"
        )
        | model
        | StrOutputParser()
    )

    # Attempt to parse or fix the text up to 3 times
    for _ in range(3):
        try:
            return parser.parse(text)  # Attempt to parse the JSON
        except Exception as e:
            # Invoke the fixing chain with the error details
            text = fixing_chain.invoke({"input": text, "error": str(e), "format_instructions": parser.get_format_instructions()}, config)
    return "Failed to parse after 3 attempts"

# Define a callback and invoke the parsing/fixing function
with get_openai_callback() as cb:
    corrected_data = '{"project_name123": "Highway Expansion", "benefits": [500000.0, 300000.0, 200000.0], "costs": [300000.0, 200000.0, 100000.0], "discount_rate": 0.1, "project_duration": 5, "net_present_value": 500000.0, "internal_rate_of_return": 0.15}'

    output = RunnableLambda(parse_or_fix_economics).invoke(
        faulty_data,
        {"tags": ["EconomicsEngineering", "CBA"], "callbacks": [cb]},
    )

    print("Corrected Output:", output)
    print("Callback Metrics:", cb)

Corrected Output: {'project_name': 'Highway Expansion', 'benefits': [500000], 'costs': [200000], 'discount_rate': 0.1, 'project_duration': 5, 'net_present_value': 300000.0, 'internal_rate_of_return': 0.15}
Callback Metrics: Tokens Used: 746
	Prompt Tokens: 669
		Prompt Tokens Cached: 0
	Completion Tokens: 77
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.00014655


##7. Streaming

In [26]:
from typing import Iterator, List
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# Initialize the model
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp')

# Define the prompt for Economics Engineering concepts
prompt = ChatPromptTemplate.from_template(
    "Write a comma-separated list of 5 economic concepts similar to: {concept}. Do not include numbers."
)

# Create a chain to process the response
str_chain = prompt | model | StrOutputParser()

# This custom parser splits an iterator of LLM tokens into a list of strings separated by commas
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    # Hold partial input until we get a comma
    buffer = ""
    for chunk in input:
        # Add current chunk to buffer
        buffer += chunk
        # While there are commas in the buffer
        while "," in buffer:
            # Split buffer on comma
            comma_index = buffer.index(",")
            # Yield everything before the comma
            yield [buffer[:comma_index].strip()]
            # Save the rest for the next iteration
            buffer = buffer[comma_index + 1 :]
    # Yield the last chunk
    yield [buffer.strip()]

# Extend the chain to include the custom parser
list_chain = str_chain | split_into_list

# Example usage for streaming chunks of similar economic concepts
for chunk in list_chain.stream({"concept": "supply and demand"}):
    print(chunk, flush=True)

# Example usage for invoking and getting the final result as a list
result = list_chain.invoke({"concept": "supply and demand"})
print("\nFinal List:", result)

['scarcity and choice']
['opportunity cost']
['inflation and deflation']
['market equilibrium']
['gross domestic product and national income']

Final List: ['scarcity and choice', 'opportunity cost', 'inflation and deflation', 'gross domestic product', 'fiscal and monetary policy']


###Async mode:

In [27]:
import asyncio

In [29]:
from typing import AsyncIterator


async def asplit_into_list(
    input: AsyncIterator[str],
) -> AsyncIterator[List[str]]:  # async def
    buffer = ""
    async for (
        chunk
    ) in input:  # `input` is a `async_generator` object, so use `async for`
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]


list_chain = str_chain | asplit_into_list

async for chunk in list_chain.astream({"concept": "supply and demand"}):
    print(chunk, flush=True)

['inflation and deflation']
['scarcity and abundance']
['opportunity cost and benefit']
['specialization and trade']
['fiscal and monetary policy']
